In [2]:
import gym
import gym_gridworld
from datetime import datetime

# DR TRPO related files
from grid_train_helper import *
from value import NNValueFunction
from utils import Logger
from grid_dr_policy import DRPolicyKL, DRPolicyWass

# ODRPO - KL

## No human 

In [4]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 10)
gamma = 0.9
lam = 1
total_eps = 5
batch_eps = 1
max_steps = 20 # max steps per episode
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 10, h2: 7, h3: 5, lr: 0.00378
***** Episode 1, Mean Return = -4.5, Mean Discounted Return = -1.8 *****
ExplainedVarNew: -0.00162
ExplainedVarOld: -0.00747
ValFuncLoss: 1.4


***** Episode 2, Mean Return = -6.5, Mean Discounted Return = -2.4 *****
ExplainedVarNew: -0.0599
ExplainedVarOld: -0.0109
ValFuncLoss: 2.06


***** Episode 3, Mean Return = -6.9, Mean Discounted Return = -2.8 *****
ExplainedVarNew: -4.32e-05
ExplainedVarOld: -2.26e-05
ValFuncLoss: 0.689


***** Episode 4, Mean Return = -6.5, Mean Discounted Return = -3.0 *****
ExplainedVarNew: -4.66e-08
ExplainedVarOld: -5.37e-06
ValFuncLoss: 0.849


***** Episode 5, Mean Return = -4.1, Mean Discounted Return = -1.9 *****
ExplainedVarNew: -9.07e-07
ExplainedVarOld: -2.93e-07
ValFuncLoss: 0.73




## Human in the Loop

In [5]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 10)
gamma = 0.9
lam = 1
total_eps = 5
batch_eps = 1
max_steps = 20 # max steps per episode
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        
        # human modifies the advantage
        # left red room
        all_advantages[0][2] += 10
        all_advantages[5][2] += 10
        all_advantages[10][1] += 10
        all_advantages[15][0] += 10
        all_advantages[20][0] += 10
        
        all_advantages[11][1] += 10
        
        # middle blue room
        all_advantages[2][2] += 10
        all_advantages[7][2] += 10
        all_advantages[12][1] += 10
        all_advantages[17][0] += 10
        all_advantages[23][0] += 10
        
        all_advantages[13][1] += 10
        
        
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 10, h2: 7, h3: 5, lr: 0.00378
***** Episode 1, Mean Return = -6.1, Mean Discounted Return = -2.4 *****
ExplainedVarNew: -0.0258
ExplainedVarOld: -0.076
ValFuncLoss: 0.928


***** Episode 2, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -1.24e-07
ExplainedVarOld: -0.000196
ValFuncLoss: 77.2


***** Episode 3, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -4.64e-05
ExplainedVarOld: -1.24e-07
ValFuncLoss: 54.1


***** Episode 4, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -1.02e-07
ExplainedVarOld: -4.64e-05
ValFuncLoss: 36.6


***** Episode 5, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -4.53e-08
ExplainedVarOld: -1.02e-07
ValFuncLoss: 28.7




# ODRPO - Wasserstein

## No human 

In [10]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 10)
gamma = 0.9
lam = 1
total_eps = 5
batch_eps = 1
max_steps = 20 # max steps per episode
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 10, h2: 7, h3: 5, lr: 0.00378
***** Episode 1, Mean Return = -6.9, Mean Discounted Return = -2.7 *****
ExplainedVarNew: -0.537
ExplainedVarOld: -0.281
ValFuncLoss: 1.56


***** Episode 2, Mean Return = -7.3, Mean Discounted Return = -3.2 *****
ExplainedVarNew: -0.00394
ExplainedVarOld: -0.0108
ValFuncLoss: 0.573


***** Episode 3, Mean Return = -8.9, Mean Discounted Return = -3.2 *****
ExplainedVarNew: -0.0059
ExplainedVarOld: -0.00282
ValFuncLoss: 1.06


***** Episode 4, Mean Return = 7.6, Mean Discounted Return = 1.6 *****
ExplainedVarNew: -3.91e-05
ExplainedVarOld: -2.8e-06
ValFuncLoss: 32.9


***** Episode 5, Mean Return = -2.9, Mean Discounted Return = -1.2 *****
ExplainedVarNew: -0.0583
ExplainedVarOld: -0.0143
ValFuncLoss: 6.37




## Human in the Loop 

In [11]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 10)
gamma = 0.9
lam = 1
total_eps = 5
batch_eps = 1
max_steps = 20 # max steps per episode
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        
        # human modifies the advantage
        # left red room
        all_advantages[0][2] += 10
        all_advantages[5][2] += 10
        all_advantages[10][1] += 10
        all_advantages[15][0] += 10
        all_advantages[20][0] += 10
        
        all_advantages[11][1] += 10
        
        # middle blue room
        all_advantages[2][2] += 10
        all_advantages[7][2] += 10
        all_advantages[12][1] += 10
        all_advantages[17][0] += 10
        all_advantages[23][0] += 10
        
        all_advantages[13][1] += 10
        
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 10, h2: 7, h3: 5, lr: 0.00378
***** Episode 1, Mean Return = -6.9, Mean Discounted Return = -2.3 *****
ExplainedVarNew: -1.17e-05
ExplainedVarOld: -0.00017
ValFuncLoss: 2.34


***** Episode 2, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -0.000336
ExplainedVarOld: -3.53e-06
ValFuncLoss: 83.4


***** Episode 3, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -4.39e-05
ExplainedVarOld: -0.000336
ValFuncLoss: 46.7


***** Episode 4, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -3.17e-07
ExplainedVarOld: -4.39e-05
ValFuncLoss: 35.5


***** Episode 5, Mean Return = 9.7, Mean Discounted Return = 7.0 *****
ExplainedVarNew: -1.51e-07
ExplainedVarOld: -3.17e-07
ValFuncLoss: 28.2


